# Penjelasan sedikit
MiniLM itu pake teknik knowledge distilation buat ngecilin model transformer gede tapi ga kureng performnya. Adapun yg dia lakukan itu dengan skema:
- Deep self-att distilation: Jadi kalo bert distil si layernya, si MiniLM dia "belajar" self att dari layer terakhirnya aja
- Self att value-relation transfer: Selain distribusi si attentionnya (dot prod query.key) dia juga mmprhitungkan hubungan antar values dalam self-att
- Teacher assistant: ini fixing transfer knowledge dari model teacher yg gede (model perantara ukuran sedang) 

$$
    \text L_{kd} = \sum_{x\in D} \text L(f_S(x),f_T(x))
$$

itu loss distilasinya, si $ \text f_S(x) \ $  adalah represent student model  $ \text f_T(x) $ adalah teachernya. Terus L-nya lossnya sih bisa kek MSe atau KL Divergence

Nah kek yg biasa kita tau kan kalo di transformer itu kita rumusin dg <br>
$$
    \text Attention(Q,K,V) = \text softmax(\frac {Q.K^T} {\sqrt {d_k}}) V
$$

si MiniLM itu disini ngeoptimize dengan cara niru self-attnya dengan dua teknik:
1. Self att distribution transfer <br>
Jadi dia niru pake KL-Divergence
$$
    \text L_{AT} = \frac {1}{A_h|x|} \sum_{a=1}^{A_h} \sum_{t=1}^{|x|} D_{KL} (A^T_{L,a,t} || A^S_{M,a,t})
$$
2. Self att value relation transfer <br>
Jadi slain distribusi att, MiniLm tu juga niru hubungan antar values mnggunakan dot product sm values
$$
    \text V RT_{L} = \text sofmax (\frac{V_L V_L^T} {\sqrt{d_k}})
$$
Lossnya habis itu diitung dengan KL-div sm kek yg si distribution transfer
$$
    \text L_{VR} = \frac {1}{A_h|x|} \sum_{a=1}^{A_h} \sum_{t=1}^{|x|} D_{KL} (VRT_{L,a,t} || VRT_{M,a,t})
$$
Nah baru total loss distilasi keduanya digabungin deh jadi $ \text L = L_{AT} + L_{VR} $

ket:
$$
    \text A_h : attention headnya,
    \text |x| = panjang tokennya,
    \text a =  head,
    \text t = token
$$

### Perumusannya dapat ditulis dalam code sbg berikut bro

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
# ini yg distribution transfernya
class SelfAttDistillation(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self,teacher_attn,student_attn):
        loss = F.kl_div(
                        F.log_softmax(student_attn,dim=-1),
                        F.softmax(teacher_attn,dim=-1),
                        reduction = 'batchmean'
                        )
        return loss
    
# ini yg self att value relation transfer
class ValueRelationDistillation(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, teacher_val, student_val):
        teacher_relation = F.softmax(torch.bmm(teacher_val, teacher_val.transpose(1,2)) /
                                     math.sqrt(teacher_val.size(-1)),dim=-1)
        student_relation = F.softmax(torch.bmm(student_val, student_val.transpose(1,2)) /
                                     math.sqrt(student_val.size(-1)),dim=-1)
        loss = F.kl_div(
            torch.log(student_relation), teacher_relation, reduction='batchmean'
        )
        return loss
    
# ini buat nyari total lossnya
class MiniLMDistillation(nn.Module):
    def __init__(self):
        super().__init__()
        self.attn_distillation = SelfAttDistillation()
        self.value_distillation = ValueRelationDistillation()

    def forward(self, teacher_attn, student_attn, teacher_val, student_val):
        loss_attn = self.attn_distillation(teacher_attn, student_attn)
        loss_value_relation = self.value_distillation(teacher_attn, student_attn)

        return loss_attn+loss_value_relation
